# Raster data

In [ ]:
# import modules
import rasterio
import rasterio.mask
from rasterio.plot import reshape_as_image 
from rasterio.plot import reshape_as_raster
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
import branca
import folium

## Rasterio

Once we have raster data on our computer, we can open them in Python using `rasterio`. Rasterio reads and writes geospatial raster datasets by interfacing Python with [Geospatial Data Abstraction Library (GDAL)](https://en.wikipedia.org/wiki/GDAL) software library. `rasterio` functions typically accept and return Numpy ndarrays, and is designed to make working with geospatial raster data more productive and more fun.

> Note: `rasterio` is not a [geographic information system (GIS)](https://blog.mapbox.com/rasterio-gdal-especially-for-python-programmers-553dabf7dd71) sofware. GIS are full-fledged computer system for capturing, storing, checking, and displaying data related to positions on Earth’s surface.

Let's start exploring the functionalities of `rasterio` by opening a GeoTIFF file withdata on global flood risk. Flood risk takes values from 0 to 5.

In [ ]:
filename = 'Data/case_study/geonode__fl1010irmt.tif'
dataset = rasterio.open(filename)
type(dataset)

The `rasterio.open()` function takes a path string or path-like object and returns an opened `DatasetReader` object. The path may point to a file of any supported raster format. Rasterio will open it using the proper GDAL format driver. Dataset objects have some of the same attributes as Python file objects.

In [ ]:
dataset.name

Properties of the raster data stored in the example GeoTIFF can be accessed through attributes of the opened dataset object. Dataset objects have bands/layers and this example has a band count of 1 since this dataset has only one raster layer.

In [ ]:
dataset.count

A dataset band is an array of values representing the partial distribution of a single variable in 2-dimensional (2D) space. All band arrays of a dataset have the same number of rows and columns.

In [ ]:
print(dataset.width)
print(dataset.height)

One can always display the *metadata* of the raster dataset for a summary of important information.

In [ ]:
dataset.meta

## Raster georeferencing

A GIS raster dataset is different from an ordinary image; its cells (or “pixels”) are mapped to regions on the Earth’s surface. Every pixel of a dataset is contained within a spatial bounding box expressed in terms of `crs` coordinates.  

In [ ]:
dataset.bounds

In [ ]:
dataset.crs

## Accessing raster data
Data from a raster band can be accessed by the band’s index number. Following the GDAL convention, bands are indexed from 1 (and not 0 as in most python arrays)

In [ ]:
band1 = dataset.read(1)
band1.shape

The `read()` method returns a Numpy N-D array.

In [ ]:
band1

In [ ]:
type(band1)

Instead of reading single bands, all bands of the input dataset can also be read into a 3-dimensional `ndarray`.

In [ ]:
bands = dataset.read()
bands.shape

The interpretation of a 3-dimension array read from rasterio is `(bands, rows, columns)` while image processing software such as `scikit-image`, `pillow` or `matplotlib` generally consider `(rows, columns, bands)`, where number of rows defines the dataset’s height and the columns are the dataset’s width.

`rasterio` provides a way to efficiently swap the axis order and you can use the following reshape functions to convert between raster and image axis order:

In [ ]:
# Convert to image
image = rasterio.plot.reshape_as_image(bands)
print(image.shape)

# Convert back to raster
raster = rasterio.plot.reshape_as_raster(image)
print(raster.shape)

Irrespective to the ordering, values from the array can be addressed by their `band, row, column` index. Notice that now indexing starts again at 0 as in classic Python arrays.

In [ ]:
bands[0, 150, 100]

## Analyzing & plotting the data

Once the data are downloaded and opened, we can start using it and analyzing it like classical numpy arrays.



In [ ]:
plus1 = bands[0,:,:]+1
plus1

We can of course plot the data using `matplotlib` functionalities

In [ ]:
plt.imshow(raster[0,:,:]);

To understand the data distribution, you can plot a histogram of the band values.

In [ ]:
f, ax = plt.subplots(1, figsize=(6,4))
ax.hist(raster[0,:,:].flatten(), color='red', alpha=0.3, label='R')
ax.legend(frameon=False, fontsize=10)
ax.set_xlabel('Data []')
ax.set_ylabel('Probability density by count');

All standard `numpy` functions apply to the raster data sets

In [ ]:
np.mean(raster)

In [ ]:
np.max(raster)

In [ ]:
np.min(raster)

You can also apply these functions along specific axes:

In [ ]:
np.mean(raster,axis=2)

## Masked rasters

It is common to *mask* raster data to restrict the analysis to valid data points or consider only some parts of the dataset.

Simple masking operations can be carried out on the raster data by exploiting `numpy`. The code below limits the visualization of the flood risk only to the areas with highest risk (>3). The mask is computed as a binary `ndarray`. Multiplication performs the masking.  

In [ ]:
mask = bands>3
plt.imshow((bands*mask).squeeze());
plt.title('Areas with critical flood risk');

Using `geopandas` we can perform masking based from geometries. 

For instance, let's consider the following image concerning cloud coverage with 718 rows and 791 columns of pixels. Each pixel has three 8-bit (uint8) channels or bands. It has a trapezoid of image data within a rectangular background of 0,0,0 value pixels.

In [ ]:
raster_src= rasterio.open("Data/raster/RGB.byte.tiff")
rasterio.plot.show(raster_src);

We now load a box shape to use as mask for the image loaded before. Everything outside the box will be made invalid.

In [ ]:
shapes = gpd.read_file('Data/raster/box.shp')
shapes.geometry[0]

In [ ]:
out_image, out_transform = rasterio.mask.mask(raster_src, shapes['geometry'], crop=False);
rasterio.plot.show(out_image);

Setting `crop=True` will limit the image only to valid pixels, e.g. the extent of the raster is also set to be the extent of the features in the shapefile.

In [ ]:
# we ignore the second output from the mask operation
out_image, _ = rasterio.mask.mask(raster_src, shapes['geometry'], crop=True);
rasterio.plot.show(out_image);

Extra information on masking is provided on [rasterio manuals on masks](https://rasterio.readthedocs.io/en/latest/topics/masks.html) [and masking](https://rasterio.readthedocs.io/en/latest/topics/masking-by-shapefile.html).

## Exercise

### Warming up

You are given a raster dataset for the global flood risk. Flood risk values are from 0 to 5. We have already explored this data in this notebook; lets load it again.

In [ ]:
filename = 'Data/case_study/geonode__fl1010irmt.tif'
flood_raster = rasterio.open(filename)

In the previous exercises, we look at the provincial boundaries of Vietnam defined using Polygons. Lets load these data again as well. 

In [ ]:
# load geospatial dataset on Vietnam provinces
gdf_vietnam = gpd.read_file('./Data/case_study/vietnam_bound.geojson')
gdf_vietnam = gdf_vietnam[['VARNAME_1','ENGTYPE_1','geometry']]
gdf_vietnam.columns = ['name','type','geometry']

We will be producing *flood risk maps* using this geometrical data together with the raster data, therefore we need to check if the CRS is the same for both datasets.

In [ ]:
print(gdf_vietnam.crs)
print(flood_raster.crs)

Lets use the mask function to first extract the flood risk data for Vietnam from the global flood risk data. We use mask between global raster data `dataset` and all provincial boundaries of Vietnam `gdf_vietnam` for this using `crop=True`

In [ ]:
out_image, _ = rasterio.mask.mask(flood_raster, gdf_vietnam['geometry'], crop=True)
rasterio.plot.show(out_image);

Now, lets use the provincial dataset and flood risk raster data to identify provinces that are vulnerable to flooding. 

For this, we first need to assign a flood risk value per province. This can be done in several ways. In this workshop, we take the mean of the raster values which is within the boundaries of the province and assign that as the flood risk of a province

In [ ]:
# iterate over each province.
all_prov_names = gdf_vietnam['name'].unique()
for prov_name in all_prov_names:
    # filter geometry of the specific province
    prov_geom = gdf_vietnam[gdf_vietnam['name'] == prov_name]['geometry']    
    # extract the raster data for that province using the mask
    prov_raster, _  = rasterio.mask.mask(flood_raster, prov_geom, crop=True)
    # assign the average value of the bounded raster as the average value of the province
    gdf_vietnam.loc[gdf_vietnam['name'] == prov_name, 'avg'] = np.mean((prov_raster[0]))

In [ ]:
# simple plot to show the mean value per province
gdf_vietnam.plot(column='avg', cmap='hot', legend=True);

For better visualization, let us plot final results using a `folium` Cloropeth .

In [ ]:
# create learn colormap interpolating 3 colors
colors = branca.colormap.LinearColormap(
    ['green', 'yellow', 'red'], vmin=gdf_vietnam.avg.min(), vmax=gdf_vietnam.avg.max())

# define style function
def raster_choropleth(row):
    return {
        "fillColor": colors(row['properties']['avg']),
        "color": "white",
        "weight": 1,
        "fillOpacity": 0.75,
    }

# create base map
# The map is centered on Hanoi, e.g. `latitude = 21.03 N`, `longitude = 105.8 E`
poly_map = folium.Map(
    location=[21.03, 105.8],    
    zoom_start=5
)

# overlay choropleth
gjson = folium.features.GeoJson(
    gdf_vietnam,
    style_function=raster_choropleth,
    ).add_to(poly_map)

# add Tooltip
folium.features.GeoJsonTooltip(
    fields=['avg'],
    aliases=['Avg. Flood Risk']
).add_to(gjson)

# add colormap to the map
poly_map.add_child(colors)

# display
poly_map

### Your turn now! :D

Your task is to follow what done before in order to compute and visualize flood risk for the **roads** of Vietnam. 

Remember that the Vietnam roads' dataset is located in `./Data/case_study/vietnam_roadnet.geojson`.

Using `geopandas`, `rasterio`, and `folium` as shown in this workshop you have to:

1. Find the average flood risk value for each road segment, identified by the LineString geometries. 
    1. For this, use the index of the geopandas dataframe as the unique id of the road segment
    2. Mask the raster data with the road LineString data
    3. Compute the average flood risk per road segment
    
2. Visualise the flood risk per road segment
    1. The map is centered on Hanoi, e.g. `latitude = 21.03 N`, `longitude = 105.8 E`;
    2. The map shows the flood risk of each road segment; 
    3. Colors are assigned using a linear `branca` colormap scaled between the minimum and the maximum flood risk;
    4. The colorbar is visible on the map;
    5. The map has a `Tooltip` feature that shows the average flood risk for each road.
    
> Hint: make sure the raster and vector datasets have the same crs!

## Solution

In [ ]:
""" Your code here"""